In [6]:
import rasterio
import rasterio
from rasterio.windows import Window

#!/usr/bin/env python3
import sys
import re
import subprocess
from pathlib import Path
import argparse
import subprocess
import sys
import re
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# paste this into a .py file or run in VS Code interactive / notebook
import subprocess
import re
import sys
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

# ---------- EDIT THESE PATHS IF NEEDED ----------
src_path = Path(r"D:/2_Analytics/9_LULC_classification/demo_ortho/KRB_BGD_ORTHO_3CM_UTM_GEOTIFF.tif")
out_prefix = Path(r"D:/2_Analytics/9_LULC_classification/demo_ortho/orthomosaic_band")
out_format = "GTiff"   # or "PNG"
workers = 1            # set >1 to run gdal_translate in parallel
compress_tiff = True
# ------------------------------------------------

def run_cmd(cmd):
    subprocess.run(cmd, check=True)

def detect_band_count(input_path):
    out = subprocess.check_output(["gdalinfo", str(input_path)], stderr=subprocess.DEVNULL).decode("utf-8", "ignore")
    bands = re.findall(r"^\s*Band\s+(\d+)", out, flags=re.MULTILINE)
    if not bands:
        raise RuntimeError("Could not detect bands with gdalinfo.")
    return max(int(b) for b in bands)

def build_translate_cmd(input_path, band_index, out_file, out_format, compress=True):
    cmd = ["gdal_translate", "-quiet", "-b", str(band_index)]
    if out_format.upper() in ("GTIFF", "TIFF", "TIF"):
        cmd += ["-co", "TILED=YES"]
        if compress:
            cmd += ["-co", "COMPRESS=LZW"]
        cmd += ["-co", "BIGTIFF=IF_SAFER"]
    if out_format:
        cmd += ["-of", out_format]
    cmd += [str(input_path), str(out_file)]
    return cmd

def main():
    # check input exists
    if not src_path.exists():
        print(f"Input file not found: {src_path}", file=sys.stderr)
        return

    # check gdal on PATH
    try:
        subprocess.run(["gdalinfo", "--version"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        subprocess.run(["gdal_translate", "--version"], check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    except Exception:
        print("Error: gdalinfo or gdal_translate not found on PATH. Install GDAL and ensure it's in PATH.", file=sys.stderr)
        return

    try:
        band_count = detect_band_count(src_path)
    except Exception as e:
        print("Failed to detect band count:", e, file=sys.stderr)
        return

    print(f"Detected {band_count} band(s). Output prefix: {out_prefix}. Output format: {out_format}. Workers: {workers}")

    jobs = []
    fmt_upper = out_format.upper()
    for b in range(1, band_count + 1):
        if fmt_upper in ("GTIFF", "GTIF", "TIFF", "TIF"):
            ext = ".tif"
        elif fmt_upper == "PNG":
            ext = ".png"
        elif fmt_upper in ("JPEG", "JPG"):
            ext = ".jpg"
        else:
            ext = ".out"
        out_file = Path(f"{out_prefix}_{b}{ext}")
        cmd = build_translate_cmd(src_path, b, out_file, out_format, compress=compress_tiff)
        jobs.append((b, out_file, cmd))

    # run jobs (serial or parallel)
    if workers <= 1:
        for b, out_file, cmd in jobs:
            print(f"Writing band {b} -> {out_file}")
            try:
                run_cmd(cmd)
            except subprocess.CalledProcessError as e:
                print(f"gdal_translate failed for band {b}: {e}", file=sys.stderr)
                return
    else:
        with ThreadPoolExecutor(max_workers=workers) as ex:
            futures = {ex.submit(run_cmd, cmd): (b, out_file) for (b, out_file, cmd) in jobs}
            for fut in as_completed(futures):
                b, out_file = futures[fut]
                try:
                    fut.result()
                    print(f"Wrote band {b} -> {out_file}")
                except subprocess.CalledProcessError as e:
                    print(f"gdal_translate failed for band {b}: {e}", file=sys.stderr)
                    return

    print("All done.")

if __name__ == "__main__":
    main()



Detected 4 band(s). Output prefix: D:\2_Analytics\9_LULC_classification\demo_ortho\orthomosaic_band. Output format: GTiff. Workers: 1
Writing band 1 -> D:\2_Analytics\9_LULC_classification\demo_ortho\orthomosaic_band_1.tif
